In [ ]:
import base64
import os
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [5]:
assert os.environ["OPENAI_API_KEY"]

In [ ]:
def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
def create_image_message(base64_image: str) -> List[dict]:
    """画像メッセージを作成する"""
    return [
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
        }
    ]

In [ ]:
class ImageAnalysisChain:
    def __init__(self, api_key: str = None, model: str = "gpt-4o"):
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.model = ChatOpenAI(model=model, temperature=0, openai_api_key=self.api_key)

    def analyze_image(
        self, image_path: str, prompt: str, max_tokens: int = 1000
    ) -> str:
        base64_image = encode_image_to_base64(image_path)
        image_message = create_image_message(base64_image)

        messages = [
            SystemMessage(content="You are a helpful assistant that analyzes images."),
            HumanMessage(content=[{"type": "text", "text": prompt}, *image_message]),
        ]

        response = self.model.invoke(messages, max_tokens=max_tokens)
        return response.content

In [12]:
chain = ImageAnalysisChain()

image_path = "./acetic_acid_dimer.png"
prompt = "この画像について詳しく説明してください。何が写っていて、どのような状況か分析してください。"

result = chain.analyze_image(image_path, prompt)

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result))

この画像は、化学分子の3Dモデルを示しています。左側の分子はアセト酢酸エステル（エチルアセトアセテート）で、右側の分子はそのエノール形態です。

- **左側の分子（ケト形）**:
  - 中央にカルボニル基（C=O）があり、酸素原子が赤色で示されています。
  - エステル基が含まれており、酸素原子が炭素と結合しています。
  - 水素原子は白色、炭素原子は灰色で示されています。

- **右側の分子（エノール形）**:
  - エノール形は、ケト形のカルボニル基がエノール基（C=C-OH）に変化したものです。
  - 二重結合があり、酸素原子が水素と結合しています。

このような分子は、ケト-エノール互変異性と呼ばれる現象を示し、化学反応や溶液中での挙動に影響を与えます。